In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [4]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")

# IA3配置

In [9]:
from peft import get_peft_model,IA3Config,TaskType

config = IA3Config(task_type=TaskType.CAUSAL_LM)
config

IA3Config(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.IA3: 'IA3'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, target_modules=None, exclude_modules=None, feedforward_modules=None, fan_in_fan_out=False, modules_to_save=None, init_ia3_weights=True)

In [10]:
model = get_peft_model(model,config)

In [11]:
model

PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(46145, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                (base_layer): Linear(in_features=2048, out_features=6144, bias=True)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 6144x1])
              )
              (dense): Linear(in_features=2048, out_features=2048, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (mlp): BloomMLP(
              (dense_

In [12]:
config

IA3Config(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.IA3: 'IA3'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path='Langboat/bloom-1b4-zh', revision=None, inference_mode=False, target_modules={'mlp.dense_4h_to_h', 'query_key_value'}, exclude_modules=None, feedforward_modules={'mlp.dense_4h_to_h'}, fan_in_fan_out=False, modules_to_save=None, init_ia3_weights=True)

In [13]:
model.print_trainable_parameters()

trainable params: 344,064 || all params: 1,303,455,744 || trainable%: 0.0264


In [14]:
args = TrainingArguments(
    output_dir="./chatbot/ia3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=3e-3#这种微调建议学习率调高点
)

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

/var/folders/vt/2rwrwr1j1hn9724wbbsr7jm80000gn/T/ipykernel_1871/1988846643.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.650300
20,2.562100
30,2.550900
40,2.459900
50,2.415700
60,2.430400
70,2.306200
80,2.389900
90,2.219700
100,2.351100


KeyboardInterrupt: 

In [18]:
import torch
device = "mps"
model = model.to(device)
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)
 

'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧可分为两个层次：一是针对考试本身，考生需要了解大纲相关的信息，同时熟悉考试的各个章节，了解每道题目应该如何答对以及如何分配答题时间；二是针对考生的具体考试情况，即考试当天的情况。在考试当天，需要考生准备充足的知识储备，合理地分配答题时间，以保证能够正确答对所有的考试题目。'